## Overall structure to understand what the hell is going on

In [1]:
import numpy as np
import os, sqlite3, pickle, sys, gzip, shutil, time
from tqdm import tqdm
import os.path as osp

from pandas import read_sql, concat
from sklearn.preprocessing import normalize, RobustScaler
from sklearn.neighbors import kneighbors_graph as knn
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
from spektral.data import Dataset, Graph
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU') 
if len(gpu_devices) > 0:
    print("GPU detected")
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('No GPU detected')

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import load_model
from spektral.data import DisjointLoader, BatchLoader
from importlib import reload

No GPU detected


##  Dataloading

In [47]:
import data_load as dl
reload(dl)
dataset=dl.graph_w_edge2()

idx_lists = dataset.index_lists
# Split data
dataset_train = dataset[idx_lists[0]]
dataset_val   = dataset[idx_lists[1]]
dataset_test  = dataset[idx_lists[2]]

loader = BatchLoader(dataset_train, batch_size=32) # the different loaders work very very differently

Loading data to memory


In [48]:
dataset.n_labels

7

### Make a Model

In [49]:
from spektral.layers.convolutional.gcn_conv import GCNConv


os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
import tensorflow as tf

from spektral.layers import ECCConv
from spektral.layers.pooling.global_pool import GlobalMaxPool 
from spektral.layers.pooling.global_pool import GlobalSumPool 

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.activations import tanh
from tensorflow.sparse import SparseTensor

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

In [50]:
class MyFirstGNN(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'softmax')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

In [51]:
model = MyFirstGNN(32, dataset.n_labels)
model.compile('adam', 'categorical_crossentropy')

In [52]:
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

Epoch 1/10
250/250 [==============================] - 4s 13ms/step - loss: -47.5794
Epoch 2/10
250/250 [==============================] - 4s 14ms/step - loss: -248.6238
Epoch 3/10
250/250 [==============================] - 4s 14ms/step - loss: -771.3811
Epoch 4/10
250/250 [==============================] - 4s 15ms/step - loss: -1716.5761: 0s - loss: -168
Epoch 5/10
250/250 [==============================] - 3s 14ms/step - loss: -3085.8470: 0s - loss
Epoch 6/10
250/250 [==============================] - 4s 16ms/step - loss: -4884.6363 - ET
Epoch 7/10
250/250 [==============================] - 4s 17ms/step - loss: -6809.7529
Epoch 8/10
250/250 [==============================] - 4s 16ms/step - loss: -9169.4922
Epoch 9/10
250/250 [==============================] - 4s 14ms/step - loss: -11698.2787
Epoch 10/10
250/250 [==============================] - 3s 13ms/step - loss: -14850.7088


## Evaluating the model

In [65]:
loader = BatchLoader(dataset_test, batch_size=32)
loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))

32/32 [==============================] - 0s 10ms/step - loss: -15469.1465
Test loss: -15469.146484375


In [66]:
@tf.function(input_signature = loader.tf_signature(), experimental_relax_shapes = True)
def test_step(inputs, targets):
    predictions = model(inputs)
    targets     = tf.cast(targets, tf.float32) 
    out         = loss_func(predictions, targets)

    return predictions, targets, out

In [67]:
inputs, targets =next(loader)

In [71]:
model(inputs)

<tf.Tensor: shape=(32, 7), dtype=float32, numpy=
array([[0.00000000e+00, 0.00000000e+00, 2.46657121e-36, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.28962942e-12, 0.00000000e+00,
        1.30242120e-06, 1.48038753e-05, 9.99983907e-01],
       [0.00000000e+00, 0.00000000e+00, 9.94830310e-01, 0.00000000e+00,
        1.76646375e-20, 2.95874896e-32, 5.16965799e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.36662197e-04, 0.00000000e+00,
        1.16153211e-25, 0.00000000e+00, 9.99663353e-01],
       [0.00000000e+00, 0.00000000e+00, 5.96743338e-02, 0.00000000e+00,
        2.62215219e-22, 1.56837091e-37, 9.40325677e-01],
       [0.00000000e+00, 0.00000000e+00, 1.108223

In [70]:
help(model.test_step)

Help on method test_step in module tensorflow.python.keras.engine.training:

test_step(data) method of __main__.MyFirstGNN instance
    The logic for one evaluation step.
    
    This method can be overridden to support custom evaluation logic.
    This method is called by `Model.make_test_function`.
    
    This function should contain the mathematical logic for one step of
    evaluation.
    This typically includes the forward pass, loss calculation, and metrics
    updates.
    
    Configuration details for *how* this logic is run (e.g. `tf.function` and
    `tf.distribute.Strategy` settings), should be left to
    `Model.make_test_function`, which can also be overridden.
    
    Arguments:
      data: A nested structure of `Tensor`s.
    
    Returns:
      A `dict` containing values that will be passed to
      `tf.keras.callbacks.CallbackList.on_train_batch_end`. Typically, the
      values of the `Model`'s metrics are returned.

